# Synthesio Automation via Selenium


Author: Zening ('Ze') Duan
Date: 2020-01-22

## 1. Load Packages

In [1]:
#!pip install selenium
import time
import datetime
import re
from selenium import webdriver
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager

from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys
#from selenium.webdriver.common.action_chains import ActionChainsxiao
from utlis_V4_1 import *
from datetime import timedelta
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select

import pandas as pd


from selenium.webdriver import FirefoxOptions

import json


## 2. Run


## 2.1 Log in

In [2]:
## Synthesio credentials
path_to_json = "./cofig.json"

with open(path_to_json, "r") as handler:
    info = json.load(handler)

Username = info["Username"]
Password = info["Password"]


## Dashboard filers
Filters = {
           'Topics': ['all',
                     ],
           'Publisher categories': ['Twitter'
                                   ],
           #only support non-default filters: lang and geo for this version. Update the non_default_filters dict if needed
           'Languages': ['English'
                        ],
            
           'Countries': {'North America': ['United States'],
                         'Africa': ['Morocco', 'Algeria', 'Tunisia'],
                         'Asia': ['India', 'Japan', 'China']
                        }
          } 


Dashboard = str('Dashboard 2')#enter your dashboard name, e.g., "Dashboard 2"

 

Begin_date = {'Year': '2022', #YYYY
             'Month': '12', #from 01 to 12
             'Day': '11', #from 01 to 31
              'Hour': '00', #from 00 to 23
              'Minute': '00' #from 00 to 59
             }

End_date = {'Year':'2022' ,#YYYY
             'Month': '12', #from 01 to 12
             'Day': '20',#from 01 to 31
            'Hour': '23',#from 00 to 23
              'Minute': '59'#from 00 to 59
           }


Time_interval = float(1.0) #change it if needed

'''
Sample = {'mode':'random'
                  ,
    'fraction': float(1)
}
'''
Sample = {'mode':'head'}

Data_format = ['XLSX', 'CSV'][1]

In [3]:
driver = setup(Username, Password, Dashboard, Filters, Data_format, Sample, publishers = 1, languages = 1, all_topics = 1, countries = 0) #countries not available for now


Begin_date_obj = date_obj(Begin_date) 
End_date_obj = date_obj(End_date) 

Delta = timedelta(hours = Time_interval) 


#Run
Current_time = Begin_date_obj

while Current_time < End_date_obj:
    try:
        timeout = driver.find_element(By.XPATH, "/html/body/div[490]/div/div/section")  #check if the timeout page occurs
        if 'Session timeout' == timeout.text:
            print('Session TimeOut')
        
            if len(driver.window_handles) > 1:
                driver.close()

            driver.switch_to.window(driver.window_handles[0])
            driver.execute_script("window.open('');")
            driver.switch_to.window(driver.window_handles[1])
    
            """
            #driver = webdriver.Firefox(executable_path = '/usr/local/Cellar/python/3.7.6_1/geckodriver')
            #driver.get("https://app.synthesio.com/dashboard/#home/workspace/35944/projects/dashboard")
            #driver.execute_script('''window.open("https://rest.synthesio.com/login/v1/login?login_challenge=064ecd2f3d5e4443b83d8b7e4e89aa23#home/workspace/35944/projects/dashboard", "_blank");''') #login URL      
            """
        
            driver = resetup(driver, Username, Password, Dashboard, Filters, Data_format, Sample)# re-set up other filters
    
    except:
        pass
    
    print(strfdate(Current_time))
    time.sleep(3)
    sampling(driver, Current_time, Time_interval)
    Volume_total, Volume = dataexport(driver, Data_format, Sample)
    print('volumetotal: ' + str(Volume_total) + ', volume: ' + str(Volume))

    '''
    # This data chunk for recording data volumn info
    VolumeTablePath = '/Users/zeningduan/Desktop/Collaboration projects/Ukraine Russia project/Data/Twitter_Synthesio/' # type in path of the volume table
    data = pd.read_csv(VolumeTablePath + 'VolumeTable_Ukraine.csv')
    data['Volume_total'].loc[(data['Year'] == int(Current_time.year)) & (data['Month'] == int(Current_time.month)) & (data['Day'] == int(Current_time.day))] = volume_total
    data['Volume'].loc[(data['Year'] == int(Current_time.year)) & (data['Month'] == int(Current_time.month)) & (data['Day'] == int(Current_time.day))] = volume
    data.to_csv(VolumeTablePath, index = False)
    '''
    
    tic = time.time()
    #waiting
    time.sleep(20) #change here based on the expected volume per click. e.g., 1000s for 50k (~14 mins baseline)

    #status checking
    count = 0 
    while count < 70: #change here based on the your patience, e.g., 20 mins more
        driver, flag = dataexportcheck(driver)
        if flag == True:
            print('Next round download is ready!')
            blankclick = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div/header/div[1]/div/div[1]')
            blankclick.click()
            print('TEST!!Printed the blank area to close the download prompting.')
            time.sleep(3)
      
            break
        else:
            count += 1
            print(count)
            time.sleep(30)

    toc = time.time()
    print('This task costs ' + str(int(toc - tic)/60) + 'min.')

    Current_time += Delta
    time.sleep(5)

<selenium.webdriver.remote.webelement.WebElement (session="65f6b21e-b7ad-4cad-a7af-021acaca9541", element="3e72b6ff-dcc3-45b1-9689-4c9dd7065988")>
2022/12/11/00:00
开始啦--开始日时刻已经输入 00:00
开始日日期已经输入 2022/12/11
结束啦--结束日日期已经输入 2022/12/11
The total Volume is  42651 . Data is ready to download.
volumetotal: 42651, volume: 42651
text found
1
text found
2


KeyboardInterrupt: 

# 

In [ ]:
#How to disable the pop-up from a complete download
#https://support.mozilla.org/en-US/questions/1370262

In [1]:
test = []
len(test)

0

In [ ]:
## 以下删除

In [4]:
driver = openweb()
time.sleep(3)
login(driver, Username, Password) #input parameter
time.sleep(3)
DashButton = driver.find_element(By.XPATH, "//div[.='" + Dashboard + "']")
DashButton.click()
time.sleep(3)
data = driver.find_element(By.CSS_SELECTOR, 'a.dashboard-nav_primary__LuW46:nth-child(4)')
data.click()
time.sleep(3)
blankclick = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div/header/div[1]/div/div[1]')
blankclick.click()
time.sleep(3)

<selenium.webdriver.remote.webelement.WebElement (session="35852eae-ae0a-458e-9674-f4e3e6952386", element="c7ad5f1d-ae07-4b02-9026-5b59331c96b1")>


In [12]:
Languages =  driver.find_element(By.XPATH, '//*[@data-tracker-id="filters-button-language"]')
Languages.click()

In [11]:
Chosen_Language = driver.find_element(By.XPATH, '//*[@title="' + Filters['Languages'][0] + '"]')
Chosen_Language.click()

In [11]:
Filters['Languages']

['English']

In [16]:
Topics = driver.find_element(By.XPATH, '//*[@data-tracker-id= "filters-button-subtopic"]')
Topics.click()

In [15]:
AllTopic = driver.find_element(By.CLASS_NAME, 'Checkbox_checkbox__0JExE')
AllTopic.click()

In [20]:
Begin_date_obj = date_obj(Begin_date) 
End_date_obj = date_obj(End_date) 

Delta = timedelta(hours = Time_interval) 

#Run
Current_time = Begin_date_obj


In [21]:
Delta_1m = timedelta(minutes=1) #1 min
T = int(Time_interval)
Delta_interval = timedelta(hours=T) #1 min

In [315]:
Period = driver.find_element(By.XPATH, '//*[@data-tracker-id="filters-button-date"]')
Period.click()

In [316]:
Clear = driver.find_element(By.XPATH, '//*[@data-tracker-id="filters-clear-button"]')
Clear.click()

In [317]:
Time_start = driver.find_elements(By.XPATH, '/html/body/div[1]/div/div/div/header/div[2]/div/div/div/div/div[3]/div[2]/div[2]/div/div/div/div/section[2]/div/div[1]/div/form/div/input[2]')
Time_start[0].send_keys(Keys.BACKSPACE * 5)
Time_start[0].send_keys(strfdate(Current_time)[-5:])

In [318]:
Date_start = driver.find_elements(By.XPATH, '/html/body/div[1]/div/div/div/header/div[2]/div/div/div/div/div[3]/div[2]/div[2]/div/div/div/div/section[2]/div/div[1]/div/form/div/input[1]')
Date_start[0].send_keys(strfdate(Current_time)[:-6]) #Y/M/D

In [319]:
Time_end = driver.find_elements(By.XPATH, '/html/body/div[1]/div/div/div/header/div[2]/div/div/div/div/div[3]/div[2]/div[2]/div/div/div/div/section[2]/div/div[2]/div[1]/form/div/input[2]')
Time_end[0].send_keys(Keys.BACKSPACE * 5)
Time_end[0].send_keys(strfdate(Current_time + Delta_interval - Delta_1m)[-5:]) #H/M

In [320]:
Date_end = driver.find_elements(By.XPATH, '/html/body/div/div/div/div/header/div[2]/div/div/div/div/div[3]/div[2]/div[2]/div/div/div/div/section[2]/div/div[2]/div[1]/form/div/input[1]')
Date_end[0].send_keys(strfdate(Current_time)[:-6]) #Y/M/D

In [321]:
Blank_icon = driver.find_elements(By.XPATH, '/html/body/div/div/div/div/header/div[2]/div/div/div/div/div[3]/div[2]/div[2]/div/div/div/div/section[2]/div')
Blank_icon[0].click()

In [322]:
Period =driver.find_element(By.XPATH, '//*[@data-tracker-id="filters-button-date"]')
Period.click()

In [323]:
Volume_total, Volume = dataexport(driver, Data_format, Sample)

The total Volume is  36688 . Data is ready to download.
This task, we downloaded 36688 mentions.


In [313]:
Current_time += Delta

In [314]:
Current_time

datetime.datetime(2021, 11, 6, 16, 0)

In [251]:
driver.refresh()

In [242]:
Blank_icon

[<selenium.webdriver.remote.webelement.WebElement (session="b2261421-94d3-4d55-b31a-e3b151ad17c8", element="d0777107-334d-4db0-88cb-4de5910291e4")>]

In [407]:
Sett = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div/header/div[1]/div/div[2]/nav/div[2]/div/div/button')
Sett.click()

In [408]:
Export_Button = driver.find_element(By.XPATH,'//*[@data-tracker-id="toggle-export-modal"]')
Export_Button.click()

In [409]:
Head = driver.find_elements(By.CLASS_NAME,'Radio_radioCircle__qbZEw')[1]
Head.click()

In [24]:
Volume = driver.find_elements(By.CLASS_NAME, "ModalExport_headline__Fe1Rl")[0]
Volume.text[:-18]

'6,729'

In [25]:
Download =  driver.find_element(By.XPATH, '//*[@data-tracker-id="mention-modal-export-export-mention"]')
Download.click()

In [410]:
Current_time += Delta

In [31]:
Period = driver.find_element(By.XPATH, '//*[@data-tracker-id="filters-button-date"]')
Period.click()

In [63]:
Date_period = driver.find_elements(By.XPATH, '//*[@data-tracker-class="date-picker-date-input"]') 
Time_period = driver.find_elements(By.XPATH, '//*[@data-tracker-class="date-picker-time-input"]')     

In [56]:
Date_period[0].send_keys(strfdate(Current_time)[:-6]) #Y/M/D
time.sleep(1)

In [61]:
Time_period[0].send_keys(Keys.BACKSPACE * 5)
Time_period[0].send_keys(strfdate(Current_time)[-5:])

In [58]:
Date_period[1].send_keys(strfdate(Current_time)[:-6]) #Y/M/D
time.sleep(1)

In [64]:
Time_period[1].send_keys(Keys.BACKSPACE * 5)
Time_period[1].send_keys(strfdate(Current_time + Delta_interval - Delta_1m)[-5:]) #H/M

In [13]:
#open the Period section
Period = driver.find_element(By.XPATH, '//*[@data-tracker-id="filters-button-date"]')
Period.click()
time.sleep(5)

Clear = driver.find_element(By.XPATH, '//*[@data-tracker-id="filters-clear-button"]')
Clear.click()
time.sleep(1)


#input current time to the first & second boxes
Date_period = driver.find_elements(By.XPATH, '//*[@data-tracker-class="date-picker-date-input"]')#Locate time/date boxes 
Time_period = driver.find_elements(By.XPATH, '//*[@data-tracker-class="date-picker-time-input"]')#Locate time/date boxes   


In [15]:
Time_period[0].send_keys(Keys.BACKSPACE * 5)
Time_period[0].send_keys(strfdate(Current_time)[-5:]) #H/M

In [19]:
Time_period = driver.find_elements(By.XPATH, '//*[@data-tracker-class="date-picker-time-input"]')#Locate time/date boxes   
Time_period[0].send_keys(Keys.BACKSPACE * 5)

In [21]:
#open the Period section
Period = driver.find_element(By.XPATH, '//*[@data-tracker-id="filters-button-date"]')
Period.click()
time.sleep(5)

Clear = driver.find_element(By.XPATH, '//*[@data-tracker-id="filters-clear-button"]')
Clear.click()
time.sleep(1)


#input current time to the first & second boxes
Date_period = driver.find_elements(By.XPATH, '//*[@data-tracker-class="date-picker-date-input"]')#Locate time/date boxes 
Date_period[0].send_keys(strfdate(Current_time)[:-6]) #Y/M/D
time.sleep(1)
Time_period = driver.find_elements(By.XPATH, '//*[@data-tracker-class="date-picker-time-input"]')#Locate time/date boxes   
Time_period[0].send_keys(Keys.BACKSPACE * 5)
Time_period = driver.find_elements(By.XPATH, '//*[@data-tracker-class="date-picker-time-input"]')#Locate time/date boxes   
Time_period[0].send_keys(strfdate(Current_time)[-5:]) #H/M
time.sleep(1)

#input end time to the last two boxes

Date_period = driver.find_elements(By.XPATH, '//*[@data-tracker-class="date-picker-date-input"]')#Locate time/date boxes 

Date_period[1].send_keys(strfdate(Current_time)[:-6]) #Y/M/D
time.sleep(1)
Time_period = driver.find_elements(By.XPATH, '//*[@data-tracker-class="date-picker-time-input"]')#Locate time/date boxes
Time_period[1].send_keys(Keys.BACKSPACE * 5)
Time_period = driver.find_elements(By.XPATH, '//*[@data-tracker-class="date-picker-time-input"]')#Locate time/date boxes
Time_period[1].send_keys(strfdate(Current_time + Delta_interval - Delta_1m)[-5:]) #H/M
time.sleep(1)

#close box, apply data export button
Period = driver.find_element(By.XPATH, '//*[@data-tracker-id="filters-button-date"]')
Period.click()
time.sleep(5)

In [13]:
Search = driver.find_elements(By.CLASS_NAME, 'date-picker-time-input')
Search[0].send_keys(strfdate(Current_time)[:-6]) #Y/M/D

IndexError: list index out of range

In [16]:
Search = driver.find_elements(By.XPATH, '//*[@data-tracker-class="date-picker-time-input"]')                           
Search

[<selenium.webdriver.remote.webelement.WebElement (session="292836af-f6d6-4904-a944-c38bed33abb1", element="7deb89d4-b8bc-4bb8-8083-e6b7ad4cb0a9")>,
 <selenium.webdriver.remote.webelement.WebElement (session="292836af-f6d6-4904-a944-c38bed33abb1", element="ead84484-bf9b-4a1c-a1d7-42102c9766ac")>]

In [17]:
Search = driver.find_elements(By.XPATH, '//*[@data-tracker-class="date-picker-date-input"]')                           
Search

[<selenium.webdriver.remote.webelement.WebElement (session="292836af-f6d6-4904-a944-c38bed33abb1", element="ec4d6b2d-8739-4223-b6cc-1221481dc06d")>,
 <selenium.webdriver.remote.webelement.WebElement (session="292836af-f6d6-4904-a944-c38bed33abb1", element="4e3f2d34-d46c-47cb-adc7-860e732aa4b5")>]

In [33]:
strfdate(Current_time + Delta_interval - Delta_1m)[-5:]

'01:59'

In [27]:
Search[3].send_keys(Keys.BACKSPACE * 5)
Search[3].send_keys(strfdate(Current_time)[-5:]) #H/M

In [32]:
Search[3].send_keys(strfdate(Current_time)[:-6])#Y/M/D

In [49]:
Search[1].send_keys(Keys.BACKSPACE * 5)
Search[1].send_keys(strfdate(Current_time + Delta_interval - Delta_1m)[-5:]) #H/M

In [8]:
Sett = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div/header/div[1]/div/div[2]/nav/div[2]/div/div/button')
Sett.click()

In [9]:
Export_Button = driver.find_element(By.XPATH,'//*[@data-tracker-id="toggle-export-modal"]')
Export_Button.click()

In [12]:
Head = driver.find_elements(By.CLASS_NAME,'Radio_radioCircle__qbZEw')[1]
Head.click()

In [42]:
Volume = driver.find_elements(By.CLASS_NAME, "ModalExport_headline__Fe1Rl")[0]
Volume.text[:-18]

'11,003'

In [71]:
InputSample = driver.find_elements(By.CLASS_NAME,'Input_input__B3DxT')[0] #click ramdon sample button
InputSample.clear()

In [72]:
num = min(int(Sample.get('fraction')*volume), 50000)
InputSample.send_keys(num)
print("This task, we downloaded " + str(num) + " mentions.")

This task, we downloaded 559 mentions.


In [74]:
Csv = driver.find_elements(By.CLASS_NAME,'Radio_radioCircle__Kr41Z')[2]
Csv.click()

In [43]:
Download =  driver.find_element(By.XPATH, '//*[@data-tracker-id="mention-modal-export-export-mention"]')
Download.click()

In [80]:
src = driver.page_source
time.sleep(4)
text_found = re.search(r'Cancel export', src)
text_found

<re.Match object; span=(41678, 41691), match='Cancel export'>

In [79]:
text_found

In [19]:
checked_volume = checkvolume(driver)
        
if checked_volume > 500000: #By default to take no more than 10 perc of the cap 50K
    print("The total Volume " , str(checked_volume) , " is larger than the cap!")
else:
    print("The total Volume is ", str(checked_volume), ". Data is ready to download.")
time.sleep(5)


InputSample = driver.find_elements(By.CLASS_NAME,'Input_input__MwyQT')[0] #click ramdon sample button
InputSample.clear()
num = min(int(Sample.get('fraction')*checked_volume), 50000)
InputSample.send_keys(num)
print("This task, we downloaded " + str(num) + " mentions.")


if Data_format == 'XLSX': #if export format is xlsx
    Xlsx = driver.find_elements(By.CLASS_NAME,'Radio_radioCircle__qbZEw')[2] #four circle buttons (head, random, xlsx, and csv) on this page share same class name
    Xlsx.click()
    time.sleep(1)

if Data_format == 'CSV':
    Csv = driver.find_elements(By.CLASS_NAME,'Radio_radioCircle__qbZEw')[3] #four circle buttons (head, random, xlsx, and csv) on this page share same class name
    Csv.click()
    time.sleep(1)


Download =  driver.find_element(By.XPATH, '//*[@data-tracker-id="mention-modal-export-export-mention"]')
Download.click()

The total Volume  17067539  is larger than the cap!
This task, we downloaded 50000 mentions.


In [17]:
driver.find_elements(By.CLASS_NAME,'Radio_radioCircle__qbZEw')[0].click()

In [ ]:
html body div div.Modal_background__cEOpN div.Modal_modal__vO8HX.ModalExport_modal__e1So4 div.ModalExport_body__zYSZG div.ModalExport_format__mherg div.Radio_wrapper__vimeU.ModalExport_formatRadio__Q5Ctv div.Radio_radio__+s3QI div.



## Archive

In [3]:
driver = openweb()

In [4]:
login(driver, Username, Password) #input parameter

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="c2be951c-321c-4ea5-99dd-961ca1b91246", element="60c6ed68-397e-4d77-b3dd-913490a0738d")>


In [5]:
selectdashboard(driver, Dashboard)

We currently have  3  Dashboards:  ['CAMER', 'Russia Ukraine', 'Voter Fraud/Election Debate']
The location of the dashboard we are seeking for is: Index[ 0 ].


In [6]:
data_tab(driver)

In [7]:
blankclick = driver.find_element_by_xpath('/html/body/div[1]/div/div/div/header/div[1]/div/div[1]')
blankclick.click()

In [8]:
Publishers = driver.find_element_by_xpath('//*[@data-tracker-id= "filters-button-publisher_category"]')
Publishers.click()

In [9]:
Chosen_Publisher = driver.find_element_by_xpath('//*[@title="' + Filters['Publisher categories'][0] + '"]')
Chosen_Publisher.click()

In [10]:
Publishers = driver.find_element_by_xpath('//*[@data-tracker-id= "filters-button-publisher_category"]')
Publishers.click()

In [11]:
Languages =  driver.find_element_by_xpath('//*[@data-tracker-id="filters-button-language"]')
Languages.click()
       

In [12]:
Options = driver.find_element_by_xpath('//*[@title="' + Filters['Languages'][0] + '"]')
Options.click()


In [13]:
Languages =  driver.find_element_by_xpath('//*[@data-tracker-id="filters-button-language"]')
Languages.click()
       

In [14]:
Topics = driver.find_element_by_xpath('//*[@data-tracker-id= "filters-button-subtopic"]')
Topics.click()

In [15]:
AllTopic = driver.find_element_by_class_name('Checkbox_checkbox__HfqeR')
AllTopic.click()

In [16]:
Topics = driver.find_element_by_xpath('//*[@data-tracker-id= "filters-button-subtopic"]')
Topics.click()

In [47]:
Begin_date_obj = date_obj(Begin_date) 
End_date_obj = date_obj(End_date)

In [48]:
Begin_date_obj

datetime.datetime(2021, 12, 31, 6, 0)

In [49]:
End_date_obj

datetime.datetime(2022, 1, 5, 22, 55)

In [50]:
Delta = timedelta(hours = Time_interval) 
Delta

datetime.timedelta(seconds=21600)

In [51]:
Current_time = Begin_date_obj

In [52]:
print(strfdate(Current_time))

2021/12/31/06:00


In [46]:
Period = driver.find_element_by_xpath('//*[@data-tracker-id="filters-button-date"]')
Period.click()

In [39]:
Clear = driver.find_element_by_xpath('//*[@data-tracker-id="filters_clear_button"]')
Clear.click()

In [53]:
Search = driver.find_elements_by_tag_name('input')
Search[0].send_keys(strfdate(Current_time)[:-6]) #Y/M/D

In [54]:
Search[1].send_keys(Keys.BACKSPACE * 5)
Search[1].send_keys(strfdate(Current_time)[-5:]) #H/M

In [55]:
Search[2].send_keys(strfdate(Current_time)[:-6])#Y/M/D


In [57]:
Delta_1m = timedelta(minutes=1) #1 min
T = int(Time_interval)
Delta_interval = timedelta(hours=T) #1 min

In [58]:
Search[3].send_keys(Keys.BACKSPACE * 5)
Search[3].send_keys(strfdate(Current_time + Delta_interval - Delta_1m)[-5:])

In [59]:
Period = driver.find_element_by_xpath('//*[@data-tracker-id="filters-button-date"]')
Period.click()

In [53]:
Sett = driver.find_element_by_xpath('/html/body/div[1]/div/div/div/header/div[1]/div/div[2]/nav/div[2]/div/button')
Sett.click()

In [54]:
Export_Button = driver.find_element_by_xpath('//*[@data-tracker-id="toggle-export-modal"]')
Export_Button.click()

In [13]:
Random = driver.find_elements_by_class_name('Radio_radioCircle__Kr41Z')[1] #four circle buttons (head, random, xlsx, and csv) on this page share same class name
Random.click()

In [15]:
Volume = driver.find_element_by_class_name("ModalExport_headline__Fe1Rl")
volume = Volume.text[:-18]
volume = int(volume.replace(',', ''))
volume

13313899

In [16]:
InputSample = driver.find_element_by_class_name('Input_input__B3DxT') #click ramdon sample button
InputSample.clear()
num = min(int(Sample.get('fraction')*volume), 50000)
InputSample.send_keys(num)

In [77]:
Xlsx = driver.find_elements_by_class_name('Radio_radioCircle__Kr41Z')[3]
#four circle buttons (head, random, xlsx, and csv) on this page share same class name
Xlsx.click()

In [75]:
Random = driver.find_elements_by_class_name('Radio_radioCircle__Kr41Z')
Random

[<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="c2be951c-321c-4ea5-99dd-961ca1b91246", element="de2663a8-8a5b-4a12-9474-dbea1ab40ea6")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="c2be951c-321c-4ea5-99dd-961ca1b91246", element="6b112902-8f63-4462-9f9f-25363db9ade7")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="c2be951c-321c-4ea5-99dd-961ca1b91246", element="6b1a6cf4-76b3-449c-a025-f587dfa45d30")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="c2be951c-321c-4ea5-99dd-961ca1b91246", element="33b5f097-117b-44f6-83e6-f2af16a64f3d")>]

In [78]:
Download =  driver.find_element_by_xpath('//*[@data-tracker-id="mention-modal-export-export-mention"]')
Download.click()

In [ ]:
<div class="Radio_wrapper__qrjl4
      
      
      ModalExport_formatRadio__Q5Ctv" title="CSV"><div class="Radio_radio__TUlSa"><div class="Radio_radioCircle__Kr41Z
             " style=""></div></div><span>CSV</span></div>